In [1]:
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df_dev = pd.read_csv('../data/durecdial/dev_data_pp.csv')

In [2]:
print(df_dev)

       user_id user_profile_age_range                 user_profile_name  \
0            0      Over 50 years old               Jacky Cheung's news   
1            0      Over 50 years old               Jacky Cheung's news   
2            0      Over 50 years old               Jacky Cheung's news   
3            0      Over 50 years old               Jacky Cheung's news   
4            0      Over 50 years old               Jacky Cheung's news   
...        ...                    ...                               ...   
12652      810     Under 18 years old  Bo Huang's news; Jay Chou's news   
12653      810     Under 18 years old  Bo Huang's news; Jay Chou's news   
12654      810     Under 18 years old  Bo Huang's news; Jay Chou's news   
12655      810     Under 18 years old  Bo Huang's news; Jay Chou's news   
12656      810     Under 18 years old  Bo Huang's news; Jay Chou's news   

      user_profile_residence user_profile_accepted_food  \
0                   Jiujiang   ['Wanzhou

In [8]:
import nltk
import re
# nltk.download('stopwords')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('all')
from nltk.util import ngrams
from nltk.corpus import stopwords
from stop_words import get_stop_words

# 각 instance 별로 문장들을 어레이에 넣는다
dialogues = df_dev.groupby('user_id')
stop_words = list(get_stop_words('en'))         #About 900 stopwords
nltk_words = list(stopwords.words('english')) #About 150 stopwords
stop_words.extend(nltk_words)

def distinct_n_s_level(sentence, n):
    if len(sentence) == 0:
        return 0.0
    word_list = re.findall(r'\w+', i,flags = re.UNICODE)
    filtered_words = [word for word in word_list if word not in stopwords.words('english')]
    output = ' '.join([w for w in filtered_words if not w in stop_words])
    NGRAMS = ngrams(nltk.word_tokenize(output), 1)
    D_NGRAMS = set(NGRAMS)
    s_dngram = len(D_NGRAMS)/len(word_list)
    return s_dngram

d_array = [] # 1. 각 대화 별 s_dngram 평균을 저장한다. length = # of dialogues
ss_array = []
for key, item in dialogues:
    # per dialogue
    # 각 어레이(instance) 별로 sentence level distinct n-gram을 계산한다.
    # 계산 방법 : https://github.com/neural-dialogue-metrics/Distinct-N/blob/main/distinct_n/metrics.py#L3
    s_group = dialogues.get_group(key)['sentence']
    s_word_len = []
    # s-level dngram 저장하는 array
    s_array = []
    for i in s_group:
        s_array.append(distinct_n_s_level(i, 1))
    # dialogue-level distinct ngram : average distinct-N of a list of sentences (the dialogue)
    ss_array.append(s_array)
    d_dngram = sum(s_array)/len(s_group)
    d_array.append(d_dngram)
avg_dataset = sum(d_array)/len(d_array) # 2. 전체 데이터셋의 d_dngram 평균값. type = float
# print(d_array)

# print(avg_dataset)

In [1]:
# 유의미한 값 1. avg_dataset 변수, 2. d_array 어레이
# avg_dataset : Dialogue-level distinct n-gram들의 평균값 (float)
# d_array : Sentence-level distinct n-gram들의 평균, 한 dialogue의 평균 distinct n-gram)들의 어레이.

In [23]:
# d_array의 분포를 시각화한다.
import plotly.figure_factory as ff
import numpy as np

group_labels = ['Distinct N-gram'] # name of the dataset

fig = ff.create_distplot([d_array], group_labels, bin_size = .1)
fig.update_layout(
    autosize = False,
    width = 800,
    height = 800,
    yaxis=dict(
        title_text="Dialogue Count",
    ),
    title_text='Distribution of Dialogue-level Distinct N-gram')
fig.update_yaxes(automargin=True)
fig.show()


In [5]:
# Further : 추천의 성공여부/만족도 정보가 있는 경우 상관관계를 비교하는 차트를 만든다.